In [ ]:
!pip install google-auth google-auth-httplib2 google-auth-oauthlib google-api-python-client

In [17]:
import os
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account
import io
import pandas as pd
from googleapiclient.http import MediaIoBaseDownload
from datetime import datetime

# Google Drive APIの認証情報を設定
SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']

# 環境変数から認証情報ファイルパスを取得 (推奨)
SERVICE_ACCOUNT_FILE = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS") or '/content/opportune-ruler-347802-94d5a92e419d.json'  # 認証情報ファイルへのパス


def get_credentials(service_account_file):
    try:
        return service_account.Credentials.from_service_account_file(service_account_file, scopes=SCOPES)
    except Exception as e:
        print(f"認証エラー: {e}")
        return None


def get_drive_service(credentials):
    try:
        return build('drive', 'v3', credentials=credentials)
    except Exception as e:
        print(f"Drive API サービスの構築エラー: {e}")
        return None


def get_sheets_service(credentials):
    try:
        return build('sheets', 'v4', credentials=credentials)
    except Exception as e:
        print(f"Sheets API サービスの構築エラー: {e}")
        return None


def create_or_get_sheet(sheets_service, spreadsheet_id, sheet_name):
    """指定された名前のシートが存在しない場合は新規作成"""
    try:
        sheets_metadata = sheets_service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        sheets = sheets_metadata.get('sheets', [])
        existing_sheets = [sheet['properties']['title'] for sheet in sheets]

        if sheet_name in existing_sheets:
            print(f"既存のシート '{sheet_name}' にデータを追加します。")
        else:
            print(f"新しいシート '{sheet_name}' を作成します。")
            requests = [{
                'addSheet': {
                    'properties': {
                        'title': sheet_name
                    }
                }
            }]
            sheets_service.spreadsheets().batchUpdate(
                spreadsheetId=spreadsheet_id,
                body={'requests': requests}
            ).execute()

        return sheet_name
    except HttpError as e:
        print(f"シート作成または取得中にエラーが発生しました: {e}")
        return None


def download_and_process_excel(file_id, drive_service, sheets_service, spreadsheet_id):
    """Excelファイルをダウンロードしてスプレッドシートに転記"""
    try:
        downloaded_file = download_file(file_id, drive_service)  # ダウンロード関数を使用
        if downloaded_file is None:
            print(f"ファイルのダウンロードに失敗しました。ID: {file_id}")
            return

        # pandasを使用してExcelファイルを読み込み
        try:
            df = pd.read_excel(downloaded_file, header=None)  # ヘッダーを無視
            # 9行目以降のデータを取得 (A9から)
            data_to_append = df.iloc[8:].fillna('').values.tolist()  # NaNを空文字に変換
            data_to_append = data_to_append[:-1]  # 最後の行を削除

            # 動的にシート名を作成
            current_month = datetime.now().strftime('%m')  # 現在の月
            sheet_name = f"Sheet_{current_month}"

            # 必要に応じて新しいシートを作成
            target_sheet = create_or_get_sheet(sheets_service, spreadsheet_id, sheet_name)

            if target_sheet:
                copy_data_to_spreadsheet(sheets_service, spreadsheet_id, target_sheet, data_to_append)

        except Exception as e:
            print(f"Excelファイルの読み込み中にエラーが発生しました: {e}")
            return

    except Exception as e:
        print(f"データ処理中にエラーが発生しました: {e}")


def download_file(file_id, drive_service):
    """Google Driveからファイルをダウンロード"""
    try:
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
        fh.seek(0)
        return fh
    except HttpError as e:
        print(f"ファイルのダウンロードに失敗しました: {e}")
        return None


def copy_data_to_spreadsheet(sheets_service, spreadsheet_id, sheet_name, data):
    """データを指定したシートに転記"""
    try:
        range_to_update = f"'{sheet_name}'!A1:G10000"  # 動的なシート名を指定
        sheets_service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=range_to_update,
            valueInputOption="USER_ENTERED",
            body={"values": data}
        ).execute()
        print(f"データをシート '{sheet_name}' に転記しました。")
    except HttpError as error:
        print(f"データ転記中にエラーが発生しました: {error}")


def get_excel_files_from_folder(folder_id, drive_service):
    """指定されたフォルダからExcelファイルを取得"""
    try:
        query = f"'{folder_id}' in parents and mimeType = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'"
        results = drive_service.files().list(q=query, fields="files(id, name)").execute()
        return results.get('files', [])
    except HttpError as e:
        print(f"フォルダ内のファイル取得中にエラーが発生しました: {e}")
        return []


def process_excel_files(folder_id, spreadsheet_id):
    credentials = get_credentials(SERVICE_ACCOUNT_FILE)
    drive_service = get_drive_service(credentials)
    sheets_service = get_sheets_service(credentials)

    if not drive_service or not sheets_service or not credentials:
        return

    excel_files = get_excel_files_from_folder(folder_id, drive_service)
    if not excel_files:
        print("指定されたフォルダにはExcelファイルがありません。")
        return

    for file in excel_files:
        file_id = file['id']
        file_name = file['name']
        print(f"処理中のファイル: {file_name} (ID: {file_id})")
        download_and_process_excel(file_id, drive_service, sheets_service, spreadsheet_id)


if __name__ == '__main__':
    folder_id = '1EyPXs0qm32Now6JtLoA_rSHBzKqxHIF5'  # Google DriveフォルダID
    spreadsheet_id = '10UvW6ti3U70oiBjyzc_LclhwiXOKIbwb79fSWIFjJrU'  # Google スプレッドシートID
    process_excel_files(folder_id, spreadsheet_id)


処理中のファイル: 20241221.xlsx (ID: 1WzLUEijpbBzCLjWpwv_tORf8dfht3jao)
既存のシート 'Sheet_12' にデータを追加します。
データをシート 'Sheet_12' に転記しました。
